In [1]:
from fmri.acquisitions import SparklingAcquisition
from fmri.operators.fourier import SpaceFourier
import numpy as np
import matplotlib.pyplot as plt


from fmri.reconstructors.full import LowRankPlusSparseFMRIReconstructor

In [2]:
data_file = "/volatile/pierre-antoine/optimed/meas_MID36_2D_Sparkling_1mmiso0_3s_FID3362.dat"
bin_file = "/volatile/pierre-antoine/optimed/dim2_i_RadialIO_P0.75_N192x192_FOV0.192x0.192_Nc6_Ns2688_c30_d2__D7M9Y2021T160.bin"
shifts = np.array((-22,0))

acq = SparklingAcquisition(data_file,bin_file,
                           frame_slicer=slice(0,100),
                           shifts=shifts,
                           )

acq.kspace_data.shape, acq.kspace_loc.shape

pymapVBVD version 0.4.3
Software version: VB


Scan 1/1, read all mdhs:   0%|          | 0.00/15.4G [00:00<?, ?B/s]

read data:   0%|          | 0/600 [00:00<?, ?it/s]

/home/pc266769/.pyenv/versions/3.6.13/envs/fmri/lib/python3.6/site-packages/mri/operators/fourier/utils.py:106: UserWarning: Frequency equal to 0.5 will be put in -0.5
  warnings.warn("Frequency equal to 0.5 will be put in -0.5")


((100, 32, 80640), (80640, 2))

In [3]:
density_comp =  acq.density_comp
smaps = acq.get_smaps()

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   2 out of  32 | elapsed:    4.4s remaining:  1.1min
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    4.9s finished


In [14]:
fourier_op = SpaceFourier(acq.img_shape, acq.n_coils, acq.n_frames, samples=acq.kspace_loc, fourier_type="gpuNUFFT", density_comp=density_comp, )

print(fourier_op.shape)
print(fourier_op.img_shape)

rec = LowRankPlusSparseFMRIReconstructor(fourier_op, lowrank_thresh=1e-3, sparse_thresh=1e-6, Smaps=smaps)

[100  32 192 192]
[192 192]


In [ ]:
M,L,S = rec.reconstruct(acq.kspace_data)

 10%|█         | 3/30 [01:14<11:08, 24.78s/it]

In [ ]:
plt.rcParams['figure.figsize'] = [30,10]
fig, axs = plt.subplots(3,10)
for m,l,s,ax in zip(M[:10,:],L[:10,:],S[:10,:],axs.transpose()):
    ax[0].imshow(abs(m))
    ax[1].imshow(abs(l))
    ax[2].imshow(abs(s))
    
[ax.xaxis.set_ticks([]) for ax in axs.ravel()]
[ax.yaxis.set_ticks([]) for ax in axs.ravel()]


axs[0,0].set_ylabel('M')
axs[1,0].set_ylabel('L')
axs[2,0].set_ylabel('S')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [30, 10]
plt.figure()
Mc = M[:, 88:104,88:104]

cas=np.reshape(Mc, (Mc.shape[0],np.prod(Mc.shape[1:])))
plt.imshow(abs(cas))
plt.colorbar()

plt.figure()
Mc = L[:, 88:104,88:104]

cas=np.reshape(Mc, (Mc.shape[0],np.prod(Mc.shape[1:])))
print(cas.shape)
plt.imshow(abs(cas))
plt.colorbar()

plt.figure()
Mc = S[:, 88:104,88:104]
cas=np.reshape(Mc, (Mc.shape[0],np.prod(Mc.shape[1:])))
cas[cas == 0] = np.NaN
plt.imshow(abs(cas))
plt.colorbar()